This notebook uses the Police Public Contact Survey (PPCS) dataset: `ppcs_cc.csv`.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import w8_probit as probit
import w8_logit as logit
import w8_estimation as est
from scipy.stats import norm
from scipy.stats import t

### Load the data 

In [2]:
# Load the dataset
dat = pd.read_csv('ppcs_cc.csv')

# Inspect distribution of the target variable
print("\nDistribution of 'anyuseofforce_coded':")
print(dat['anyuseofforce_coded'].value_counts(normalize=True))

# Inspect value counts for categorical variables
categorical_vars = ["sblack", "shisp", "swhite", "sother", "smale", "omajblack", 
                    "omajhisp", "omajwhite", "omajother", "osplit", "inctype_lin", "sbehavior"]


Distribution of 'anyuseofforce_coded':
anyuseofforce_coded
0    0.994999
1    0.005001
Name: proportion, dtype: float64


Table with summary statistics

In [ ]:
# Define groups for demographic categories
group_vars = ["swhite", "sblack", "shisp", "sother"]

# List of all variables for which we want to compute means
all_vars = dat.columns

# Initialize an empty DataFrame to store results
summary_table = pd.DataFrame()

# Calculate the overall mean for each variable
overall_means = dat[all_vars].mean()
summary_table["Variable"] = all_vars
summary_table["Full Sample"] = overall_means.values

# Calculate the mean for each variable within each group
for group in group_vars:
    group_means = dat.loc[dat[group] == 1, all_vars].mean()
    summary_table[group.capitalize()] = group_means.values

# Add a row for "Number of Observations"
num_obs_row = pd.DataFrame({
    "Variable": ["Number of Observations"],
    "Full Sample": [dat.shape[0]],
    **{group.capitalize(): [dat.loc[dat[group] == 1].shape[0]] for group in group_vars}
})

# Append the "Number of Observations" row to the summary table
summary_table = pd.concat([summary_table, num_obs_row], ignore_index=True)

# Format the table for display
summary_table = summary_table.set_index("Variable")
print(summary_table)

# Optional: Save the table to a CSV for further analysis
summary_table.to_csv('grouped_summary_statistics_with_observations.csv')


summary table for assignment

In [3]:
# attach lanbels to data
labels = {
    'sblack': 'Black',
    'shisp': 'Hispanic',
    'swhite': 'White',
    'sother': 'Other race',
    'sage': 'Age',
    'sempl': 'Employed last week or not',
    'smale': 'Male',
    'spop': 'Population size of civilian\'s address',
    'sincome': 'Income (categorical)',
    'sbehavior': 'Behavior of civilian in encounter',
    'omajwhite': 'Officer unit majorly white',
    'omajother': 'Officer unit majorly other',
    'omajblack': 'Officer unit majorly black',
    'omajhisp': 'Officer unit majorly hispanic',
    'osplit': 'Officer unit split race',
    'daytime': 'Time of encounter',
    'year': 'Year',
    'inctype_lin': 'Incident type',
    'anyuseofforce_coded': 'Any use of force by officer'}

data_labeled = dat.rename(columns=labels)
data_labeled.columns
# table with summary statistics conditional on race and interaction
# a. make columns for each race and full sample
full_sample = dat
white_only = dat[dat['swhite'] == 1]
black_only = dat[dat['sblack'] == 1]
hispanic_only = dat[dat['shisp'] == 1]
other_only = dat[dat['sother'] == 1 ]
groups = [full_sample, white_only, black_only, hispanic_only, other_only]
group_names = ['Full sample', 'White only', 'Black only', 'Hispanic only', 'Other only']
variables = ['sage','sempl', 'smale','sincome', 'spop', 'sbehavior', 'omajwhite', 
             'omajblack', 'omajhisp', 'omajother', 'osplit', 'daytime', 
             'inctype_lin', 'anyuseofforce_coded']
variables_labeled = [labels[var] for var in variables]
#####################################################
# create empty dataframe to store summary statistics
summary_stats = pd.DataFrame(columns=group_names, index=variables_labeled)
# append two columns for p-values from t-tests between white vs black and hispanic vs white
summary_stats['p-value (Black vs White)'] = np.nan
summary_stats['p-value (Hispanic vs White)'] = np.nan
summary_stats.index.name = 'Variable'
for i, group in enumerate(groups):
    summary_stats.iloc[:, i] = group[variables].mean().round(2) #mean
    # t-test means of black vs white and hispanic vs white
    if group_names[i] == 'Black only':
        from scipy.stats import ttest_ind
        t_stat, p_val = ttest_ind(black_only[variables], white_only[variables], equal_var=False, nan_policy='omit')
        summary_stats['p-value (Black vs White)'] = p_val.round(2)
    if group_names[i] == 'Hispanic only':
        from scipy.stats import ttest_ind
        t_stat, p_val = ttest_ind(hispanic_only[variables], white_only[variables], equal_var=False, nan_policy='omit')
        summary_stats['p-value (Hispanic vs White)'] = p_val.round(2)

#####################################################
summary_stats.to_latex('summary_stats.tex', index=True, decimal=',', float_format="%.2f")

prepare estimations

In [4]:
# Declare labels    
y_lab = 'anyuseofforce_coded'
#x_lab = ['const', 'sblack', 'shisp', 'sother']
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior']

dat['sage'] = dat['sage'] / 10
dat['sagesq'] = dat.sage * dat.sage 

# create extra variables 
N = dat.shape[0]
dat['const'] = np.ones((N,))

# Rebuild the dataset
dat = dat[[y_lab] + x_lab].copy()

# Check for missing data
assert dat.notnull().all(axis=1).all(), 'Missing values detected. Clean your data!'

dat.tail(5)

,anyuseofforce_coded,const,sblack,shisp,sother,smale,sempl,sincome,spop,sage,sagesq
3794,0,1.0,0,0,0,0,1,3,1,7.2,51.84
3795,0,1.0,0,0,0,0,0,2,1,7.1,50.41
3796,0,1.0,0,0,0,0,0,1,1,7.6,57.76
3797,0,1.0,0,0,0,0,0,3,4,7.9,62.41
3798,0,1.0,0,0,0,0,0,2,1,7.5,56.25


In [5]:
# descriptive of who's been victim to police violence
test = dat[dat['anyuseofforce_coded']==1] 

# List of all variables for which we want to compute means
all_vars = test.columns

# Initialize an empty DataFrame to store results
summary_table = pd.DataFrame()

# Calculate the overall mean for each variable
overall_means = test[all_vars].mean()
summary_table["Variable"] = all_vars
summary_table["Full Sample"] = overall_means.values

# Format the table for display
summary_table = summary_table.set_index("Variable")
print(summary_table)

# Optional: Save the table to a CSV for further analysis
summary_table.to_latex('test.csv')


                     Full Sample
Variable                        
anyuseofforce_coded     1.000000
const                   1.000000
sblack                  0.157895
shisp                   0.315789
sother                  0.052632
smale                   0.789474
sempl                   0.473684
sincome                 2.052632
spop                    1.947368
sage                    3.078947
sagesq                 10.666842


In [6]:
# Extract y and X
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print(K)
print(np.shape(x))

10
(3799, 10)


In [7]:
count_violent_1 = (dat['anyuseofforce_coded'] == 1).sum()
print(f"Number of 1s in 'anyuseofforce_coded': {count_violent_1}")

Number of 1s in 'anyuseofforce_coded': 19


## Estimate using Probit

In [8]:
# Initialize starting values
theta0 = probit.starting_values(y, x)

# Estimate model with probit
probit_results = est.estimate(probit.q, theta0, y, x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.027097
         Iterations: 82
         Function evaluations: 1100
         Gradient evaluations: 100


In [9]:
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
probit_tab

Optimizer succeeded after 82 iter. (1100 func. evals.). Final criterion:   0.0271.
Probit, y = anyuseofforce_coded


,theta,se,t
const,-3.3340,0.6515,-5.1172
sblack,0.0972,0.2574,0.3776
shisp,0.3859,0.2098,1.8393
sother,0.0831,0.3769,0.2204
smale,0.5230,0.1992,2.6259
sempl,-0.5300,0.1964,-2.6981
sincome,0.0330,0.1041,0.3169
spop,0.2102,0.0726,2.8938
sage,0.3334,0.3739,0.8917
sagesq,-0.0653,0.0505,-1.2938


In [ ]:
# White's information matrix test for model misspecification
whites_test = probit.whites_imt_probit(probit_results['theta'], y, x)
probit.print_test_stats(whites_test[0], whites_test[1])

White's Information Matrix Test for Probit Model Misspecification
------------------------------------------------------------------
Test Statistic: 2934.0075
P-value: 0.0000
Result: Reject the null hypothesis of correct model specification at the 5% significance level.


## Estimate using Logit

In [11]:
# Initialize starting values
theta0 = logit.starting_values(y, x)

# Estimate model with logit
logit_results = est.estimate(logit.q, theta0, y, x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.027131
         Iterations: 116
         Function evaluations: 1408
         Gradient evaluations: 128


In [12]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeeded after 116 iter. (1408 func. evals.). Final criterion:  0.02713.
Logit, y = anyuseofforce_coded


,theta,se,t
const,-7.7875,1.8624,-4.1813
sblack,0.3374,0.6768,0.4986
shisp,1.0106,0.5424,1.8630
sother,0.1396,1.0524,0.1327
smale,1.3367,0.5837,2.2901
sempl,-1.4041,0.5357,-2.6211
sincome,0.0777,0.2944,0.2640
spop,0.5491,0.1832,2.9978
sage,1.1958,1.1221,1.0657
sagesq,-0.2161,0.1621,-1.3329


In [13]:
# White's information matrix test for model misspecification
whites_test = logit.whites_imt_logit(logit_results['theta'], y, x)
logit.print_test_stats(whites_test[0], whites_test[1])

White's Information Matrix Test for Probit Model Misspecification
------------------------------------------------------------------
Test Statistic: 2444.5258
P-value: 0.0000
Result: Reject the null hypothesis of correct model specification at the 5% significance level.


## Average partial effects

### Probit

In [ ]:
# Estimating the average partial effects using the probit
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother'] 
res_probit = probit.properties(x, probit_results['theta'],print_out = True,se=True,indices=indices, labels = labels)
res_probit

### Logit

In [ ]:
# Estimating the average partial effects using the logit
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother']  
res_logit = logit.properties(x, logit_results['theta'],print_out = True,se=True,indices=indices, labels = labels)
res_logit

## Partial Effects

#### Defining different fixed vectors

In [ ]:
#means of the regressors
print(f"{np.mean(dat['sage']):.2f}")
print(f"{np.mean(dat['sagesq']):.2f}")
print(f"{np.mean(dat['sincome']):.2f}")
print(f"{np.mean(dat['spop']):.2f}")

In [ ]:
# Original vector
# make vector of stereotypical white young man 
x_lab = ['const', 'sblack', 'shisp', 'sother', 
         'smale', 'sage', 'sempl', 'sincome',
         'spop', 'daytime', 'inctype_lin', 'omajblack',
         'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0,
                 1, 2.5, 0, 1,
                 4, 5, 1, 0, # daytime = 5 means the incident happened at night
                 0, 0, 1, 6.25]).reshape(1, -1)

pd.DataFrame(x_me, columns=x_lab, index=['x_me'])

#### Swiching race from white to black, hispanic and other

In [ ]:
# make vector of stereotypical white old woman
x_lab = ['const', 'sblack', 'shisp', 'sother', 
         'smale', 'sage', 'sempl', 'sincome',
         'spop', 'daytime', 'inctype_lin', 'omajblack',
         'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0,
                 0, 4.5, 1, 3,
                 2, 2, 1, 0, 
                 0, 0, 1, 20.25]).reshape(1, -1)

pd.DataFrame(x_me, columns=x_lab, index=['x_me'])


In [ ]:
"""# the average characteristics for age, agesq, income, spop
print(f"{np.mean(dat['sage']):.2f}")
print(f"{np.mean(dat['sagesq']):.2f}")
print(f"{np.mean(dat['sincome']):.2f}")
print(f"{np.mean(dat['spop']):.2f}")
print(f"{np.mean(dat['daytime']):.2f}")

x_lab = ['const', 'sblack', 'shisp', 'sother', 
         'smale', 'sage', 'sempl', 'sincome',
         'spop', 'daytime', 'inctype_lin', 'omajblack',
         'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0,
                 0, 4.1, 0, 2.16,
                 1.36, 5, 1, 0, 
                 0, 0, 1, 19.42]).reshape(1, -1)

pd.DataFrame(x_me, columns=x_lab, index=['x_me'])"""

In [ ]:
b_pr = probit_tab.theta.values
me_race_pr = probit.G(x_me2@b_pr) - probit.G(x_me@b_pr) 

In [ ]:
gx0 = norm.pdf(x_me@b_pr)
gx2 = norm.pdf(x_me2@b_pr)

grad_d_pr = gx2*x_me2 - gx0*x_me

In [ ]:
def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_d_pr = get_se(grad_d_pr, probit_results['cov'])

In [ ]:
me_dict = {'Marginal Effect': me_race_pr[0],
           's.e.':            se_d_pr}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

### Logit

In [ ]:
b_lg = logit_tab.theta.values
me_race_lg = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)

In [ ]:
# Compute the logistic function exponential terms for x_me2 and x_me
exp_x0_b = np.exp(-(x_me@b_lg))
exp_x2_b = np.exp(-(x_me2@b_lg))

grad_d_lg = (x_me2 * exp_x2_b)/ (1 + exp_x2_b)**2 - (x_me * exp_x0_b)/ (1 + exp_x0_b)**2

se_d_lg = get_se(grad_d_lg, logit_results['cov'])

In [ ]:
# find p value for marginal effects
from scipy.stats import norm
p_values = 2 * (1 - norm.cdf(np.abs(tab['t'])))
tab['p-value'] = p_values

In [ ]:
tab.to_latex('me_probit_youngman_hisp.tex')

In [ ]:
me_dict = {'Marginal Effect': me_race_lg[0],
           's.e.':            se_d_lg}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

In [ ]:
p_values = 2 * (1 - norm.cdf(np.abs(tab['t'])))
tab['p-value'] = p_values

In [ ]:
tab.to_latex('me_logit_youngman_hisp.tex')

In [ ]:
files = [
    'me_probit_youngman.tex',
    'me_probit_youngman_hisp.tex',
    'me_logit_youngman.tex',
    'me_logit_youngman_hisp.tex',
    'me_probit_old_woman.tex',
    'me_probit_old_woman_hisp.tex',
    'me_logit_old_woman.tex',
    'me_logit_old_woman.tex'
]

with open('combined_table.tex', 'w') as outfile:
    outfile.write(r'\begin{table}[ht!]' + '\n')
    outfile.write(r'\centering' + '\n')
    outfile.write(r'\caption{Marginal Effects from Probit and Logit Models}' + '\n')
    outfile.write(r'\begin{tabular}{lccc}' + '\n')
    outfile.write(r'\toprule' + '\n')
    
    panels = ['Panel A: Probit, Young Man',
              'Panel B: Probit, Young Man Hispanic',
              'Panel C: Logit, young man',
              'Panel D: Logit, young man, hispanic',
              'Panel E: Probit, Old Woman',
              'Panel F: Probit, Old Woman, hispanic',
              'Panel G: Logit, Old Woman',
              'Panel H: Logit, Old Woman, hispanic']

    for panel_name, fname in zip(panels, files):
        outfile.write(r'\midrule' + '\n')
        outfile.write(r'\multicolumn{4}{c}{\textbf{' + panel_name + r'}} \\' + '\n')
        outfile.write(r'\midrule' + '\n')
        with open(fname) as infile:
            content = infile.read()
            outfile.write(content + '\n')
    
    outfile.write(r'\bottomrule' + '\n')
    outfile.write(r'\end{tabular}' + '\n')
    outfile.write(r'\end{table}' + '\n')